In [ ]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
import tkinter as tk
from tkinter import ttk

In [ ]:
data = pd.read_csv('college_list.csv')
df = pd.DataFrame(data)
one_hot_encoded = pd.get_dummies(df[['DISTRICT', 'FACILITIES', 'COLLEGENAME']])
one_hot_encoded['RANKING'] = df['RANKING']
grouped = one_hot_encoded.groupby(df['COLLEGENAME']).sum()
grouped_binarized = grouped.drop(columns=['RANKING']).gt(0).astype(bool)  # Change to bool type
frequent_itemsets = apriori(grouped_binarized, min_support=0.001, use_colnames=True)
rules = association_rules(frequent_itemsets, metric='confidence', min_threshold=0.001)

In [ ]:
root = tk.Tk()
root.title("College Recommendation System")
root.geometry("600x600")
root.configure(bg="#f2f2f2")

In [ ]:
style = ttk.Style()
style.configure('TLabel', background="#f2f2f2", font=('Arial', 12))
style.configure('TButton', font=('Arial', 12, 'bold'), background="#4CAF50", foreground="black")  # Changed foreground to black
style.configure('TEntry', font=('Arial', 10))
style.configure('TCombobox', font=('Arial', 10))

In [ ]:
facilities_options = df['FACILITIES'].unique().tolist()
location_options = df['DISTRICT'].unique().tolist()

In [ ]:
def recommend_college():
    facilities_preference = facilities_dropdown.get()
    location_preference = location_dropdown.get().upper()
    ranking_preference = ranking_var.get()
    
    # Validate and parse ranking preference
    try:
        if '-' in ranking_preference:
            ranking_min, ranking_max = map(int, ranking_preference.split('-'))
        else:
            raise ValueError("Invalid ranking range format")
    except ValueError as e:
        result_text.set(f"Error: {e}")
        return
    
    input_features = set()
    if facilities_preference:
        input_features.add('FACILITIES_' + facilities_preference)
    if location_preference:
        input_features.add('DISTRICT_' + location_preference)
    
    eligible_colleges = grouped[(grouped['RANKING'] >= ranking_min) & (grouped['RANKING'] <= ranking_max)].index
    matching_rules = rules[rules['antecedents'].apply(lambda x: input_features.issubset(x))]
    
    recommended_colleges = set()
    for _, row in matching_rules.iterrows():
        for item in row['consequents']:
            if item.startswith('COLLEGENAME_'):
                college = item.replace('COLLEGENAME_', '')
                if college in eligible_colleges:
                    recommended_colleges.add(college)
    
    display_colleges(recommended_colleges)


In [ ]:
def display_colleges(recommended_colleges):
    if recommended_colleges:
        display_text = "Recommended Colleges:\n"
        for college in recommended_colleges:
            college_info = df[df['COLLEGENAME'] == college]
            display_text += f"{college}\nLocation: {college_info['DISTRICT'].values[0]}\nRanking: {college_info['RANKING'].values[0]}\n\n"
        result_text.set(display_text)
    else:
        result_text.set("No colleges found.")

In [ ]:
def validate_ranking(event):
    ranking_input = ranking_var.get()
    try:
        if '-' in ranking_input:
            min_rank, max_rank = map(int, ranking_input.split('-'))
            ranking_error_label.config(text="Valid range", fg="green")
        else:
            raise ValueError
    except ValueError:
        ranking_error_label.config(text="Invalid range format", fg="red")

In [ ]:
def start_progress():
    progress_bar.start()

In [ ]:
def stop_progress():
    progress_bar.stop()

In [ ]:
title_label = tk.Label(root, text="College Recommendation System", font=('Times New Roman ', 36, 'bold'), bg="#f2f2f2", fg="#333333")
title_label.pack(pady=10)

In [ ]:
frame = tk.Frame(root, bg="#ffffff", bd=10, relief=tk.RIDGE)
frame.place(relx=0.5, rely=0.15, relwidth=0.85, relheight=0.55, anchor='n')

In [ ]:
ttk.Label(frame, text="College Facilities Preference:").grid(column=0, row=0, padx=10, pady=10, sticky=tk.W)
facilities_dropdown = ttk.Combobox(frame, values=facilities_options)
facilities_dropdown.grid(column=1, row=0, padx=10, pady=10)

In [ ]:
ttk.Label(frame, text="College Location Preference:").grid(column=0, row=1, padx=10, pady=10, sticky=tk.W)
location_dropdown = ttk.Combobox(frame, values=location_options)
location_dropdown.grid(column=1, row=1, padx=10, pady=10)

In [ ]:
ttk.Label(frame, text="College Ranking Preference Range (e.g., 50-80):").grid(column=0, row=2, padx=10, pady=10, sticky=tk.W)
ranking_var = tk.StringVar()
ranking_entry = ttk.Entry(frame, textvariable=ranking_var)
ranking_entry.grid(column=1, row=2, padx=10, pady=10)
ranking_entry.bind("<KeyRelease>", validate_ranking)

In [ ]:
ranking_error_label = ttk.Label(frame, text="", background="#ffffff")
ranking_error_label.grid(column=1, row=3, padx=10, pady=5)

In [ ]:
ttk.Button(frame, text="Get Recommendations", command=lambda: [start_progress(), recommend_college(), stop_progress()]).grid(column=0, row=4, columnspan=2, pady=20)

In [ ]:
progress_bar = ttk.Progressbar(root, orient=tk.HORIZONTAL, length=400, mode='indeterminate')
progress_bar.place(relx=0.5, rely=0.72, anchor='center')

In [ ]:
result_frame = tk.Frame(root, bg="#ffffff", bd=10, relief=tk.RIDGE)
result_frame.place(relx=0.5, rely=0.75, relwidth=0.85, relheight=0.2, anchor='n')

In [ ]:
canvas = tk.Canvas(result_frame, bg="#ffffff")
canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

In [ ]:
scrollbar = ttk.Scrollbar(result_frame, orient=tk.VERTICAL, command=canvas.yview)
scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

In [ ]:
content_frame = tk.Frame(canvas, bg="#ffffff")

In [ ]:
canvas.create_window((0, 0), window=content_frame, anchor="nw")

In [ ]:
def update_scroll_region(event):
    canvas.config(scrollregion=canvas.bbox("all"))

In [ ]:
content_frame.bind("<Configure>", update_scroll_region)

In [ ]:
result_text = tk.StringVar()

In [ ]:
result_label = ttk.Label(content_frame, textvariable=result_text, background="#ffffff", font=('Arial', 12), wraplength=550)
result_label.pack(padx=10, pady=10)


In [ ]:
root.mainloop()